In [1]:
import os
import tensorflow as tf

from PIL import UnidentifiedImageError

# image processsing
from tensorflow.keras.preprocessing.image import (load_img,
                                                  img_to_array,
                                                  ImageDataGenerator)

# VGG16 model
from tensorflow.keras.applications.vgg16 import (preprocess_input,
                                                 decode_predictions,
                                                 VGG16)

# layers
from tensorflow.keras.layers import (Flatten, 
                                     Dense, 
                                     Dropout, 
                                     BatchNormalization)

# generic model object
from tensorflow.keras.models import Model

# optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD, Adam

#scikit-learn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# for plotting
import numpy as np
import matplotlib.pyplot as plt

2024-04-09 16:06:59.530667: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 16:06:59.535140: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 16:06:59.590349: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 16:07:00.833457: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
filepath = os.path.join("../../../../cds-vis-data/Tobacco3482/")
image = load_img(filepath, target_size = (224, 224))

In [42]:
filepath = os.path.join("../../../../cds-vis-data/Tobacco3482/ADVE/tob06517.41_tob0651742.jpg")
title = filepath.split("/")[-1]
print(title)
test_title = title.split(".")
print(test_title)

test_title[-1] = "." + test_title[-1]

test_title = "".join(test_title)
print(test_title)

tob06517.41_tob0651742.jpg
['tob06517', '41_tob0651742', 'jpg']
tob0651741_tob0651742.jpg


In [40]:
type(test_title[0])

str

In [48]:
# clean image titles

def title_cleaner(filepath):

    title = filepath.split("/")[-1]

    splitted_title = title.split(".")
    splitted_title[-1] = "." + splitted_title[-1] # add . before the fileformat
    splitted_title = "".join(splitted_title)
    
    return cleaned_filepath


In [39]:
folder_path = os.path.join("../../../../cds-vis-data/Tobacco3482")

# Initialize empty lists
list_of_images = [] 
list_of_labels = []

# Loop through subfolders
for subfolder in sorted(os.listdir(folder_path)):

        # Extract images from the individual folders
        subfolder_path  = os.path.join(folder_path, subfolder)

        for file in os.listdir(subfolder_path):
                
                individual_filepath = os.path.join(subfolder_path, file)

                try:
                        # Load image from file
                        image = load_img(individual_filepath, target_size = (224, 224))
                        
                        #list_of_images.append(image)

                        # Extract image name and append to list
                        #image_name = individual_filepath.split("/")[-1]
                        #list_of_images.append(image_name)

                        # Extract label and append to list
                        label = subfolder_path.split("/")[-1]
                        #list_of_labels.append(label)
                        
                        # Append the preprocessed image along with its label as a tuple
                        list_of_images.append((image, label))

                except (UnidentifiedImageError):
                        print(f"Skipping {individual_filepath}")



Skipping ../../../../cds-vis-data/Tobacco3482/ADVE/Thumbs.db
Skipping ../../../../cds-vis-data/Tobacco3482/Email/Thumbs.db
Skipping ../../../../cds-vis-data/Tobacco3482/Form/Thumbs.db
Skipping ../../../../cds-vis-data/Tobacco3482/Letter/Thumbs.db
Skipping ../../../../cds-vis-data/Tobacco3482/Memo/Thumbs.db
Skipping ../../../../cds-vis-data/Tobacco3482/News/Thumbs.db
Skipping ../../../../cds-vis-data/Tobacco3482/Note/Thumbs.db
Skipping ../../../../cds-vis-data/Tobacco3482/Report/Thumbs.db
Skipping ../../../../cds-vis-data/Tobacco3482/Resume/Thumbs.db
Skipping ../../../../cds-vis-data/Tobacco3482/Scientific/Thumbs.db


In [11]:
def load_images(folder_path):
        
        # Initialize empty lists
        list_of_images = [] 
        list_of_labels = []

        for subfolder in sorted(os.listdir(folder_path)):
                
                # Extract images from the individual folders
                subfolder_path  = os.path.join(folder_path, subfolder)
                
                for file in os.listdir(subfolder_path):
                        
                        individual_filepath = os.path.join(subfolder_path, file)
                        
                        try:
                                # Load image from file
                                image = load_img(individual_filepath, target_size = (224, 224))
                                list_of_images.append(image)

                                # Extract image name and append to list
                                #image_name = individual_filepath.split("/")[-1]
                                #list_of_images.append(image_name)

                                # Extract label and append to list
                                label = subfolder_path.split("/")[-1]
                                list_of_labels.append(label)
                
                        except (UnidentifiedImageError):
                                print(f"Skipping {individual_filepath}")

        return list_of_images, list_of_labels

In [46]:
preprocessed_images = []

for image_tuple in list_of_images:
    image = image_tuple[0]  # Extract the image from the tuple
    label = image_tuple[1]  # Extract the label from the tuple

    # Convert the image pixels to a numpy array
    image_array = img_to_array(image)

    # reshape data to 4D tensor (shape = [number of images, height, width, number of color-channels])
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

    # prepare the image for the VGG model
    image = preprocess_input(image)
    
    # append the preprocessed image along with its label as a tuple
    preprocessed_images.append(image)


In [21]:

def preprocess_images_old(list_of_images):

    # Initialize empty lists
    preprocessed_images = []

    for image_tuple in list_of_images:
        image = image_tuple[0]  # Extract the image from the tuple
        label = image_tuple[1]

        # convert the image pixels to a numpy array
        image = img_to_array(image) 

        # reshape data to 4D tensor (shape = [number of images, height, width, number of color-channels])
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        
        # prepare the image for the VGG model
        image = preprocess_input(image)

        # append to list
        preprocessed_images.append(image)

    return preprocessed_images


In [118]:
# Finetune the VGG16 model on new data

# Split data into train and test:

# Extract images and labels from the tuble into separate lists
X = np.array([image for image, label in preprocessed_images])
y = np.array([label for image, label in preprocessed_images])

# Split data into training and testing sets. 'stratify' ensures that all labels are represented in both sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 123)

# normalize
X_train = X_train.astype("float32") / 255.
X_test = X_test.astype("float32") / 255.

# one-hot encondings
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)   # shape (2785, 10)
y_test = lb.fit_transform(y_test)     # shape (697, 10)

# Tensor shape 4D (num of images, 224, 224, 3)
X_train = np.reshape(X_train, (X_train.shape[0],) + X_train.shape[2:])  # shape (2785, 224, 224, 3)
X_test = np.reshape(X_test, (X_test.shape[0],) + X_test.shape[2:])      # shape (697, 224, 224, 3)

# Scale image features
#def scale(X):
#    scaled_images  = X  / 255.0
#    return scaled_images 

Vi kører alle
```
model = VGG16()

# skal vi alle gøre - load model without classifier layers
model = VGG16(include_top = False,        # dont use the final layer of the classifier
              pooling = 'avg',
              input_shape = (32, 32, 3))  # (height, width, number of color-channels)

# skal vi alle gøre - mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False
```

1. Ida kører clean model:
```
flat1 = Flatten()(model.layers[-1].output)  # take final layer (-1) in the model and append to flat1
class1 = Dense(128, activation = 'relu')(flat1)
output = Dense(10, activation = 'softmax')(class1)
```

2. Laura kører BatchNorm
```
flat1 = Flatten()(model.layers[-1].output)
bn = BatchNormalization()(flat1)
class1 = Dense(128, 
               activation='relu')(bn)
output = Dense(10, 
               activation='softmax')(class2)
```

3. Jeg Sofie kører data argumentation - nedenstående

4. Alle kører både sgd og adam!
```
sgd = SGD(learning_rate=lr_schedule)
adam = Adam(learning_rate=lr_schedule)
```

In [ ]:
# load model without classifier layers
model = VGG16(include_top = False, 
              pooling = 'avg',
              input_shape = (224, 224, 3)) ###???? er input_shape samme som Ross (32, 32, 3) eller vores (1, 224, 224, 3)???

# mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False
    
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
bn = BatchNormalization()(flat1)
class2 = Dense(128, 
               activation='relu')(bn)
output = Dense(10, 
               activation='softmax')(class2)

# define new model
model = Model(inputs = model.inputs, 
              outputs = output)

# compile
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = 0.01,
    decay_steps = 10000,
    decay_rate = 0.9)
    
sgd = SGD(learning_rate=lr_schedule)
#adam = Adam(learning_rate=lr_schedule)

model.compile(optimizer = sgd,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
# summarize
model.summary()

In [98]:
# Define data generator

# Add noise to an image
def add_noise(img):
    noise = np.random.normal(0, scale = 25, size = img.shape)
    noisy_image = np.clip(image + noise, 0, 255) #.astype(np.uint8)
    return noisy_image

# flip along x axis (mirror image)
datagen = ImageDataGenerator(horizontal_flip = True, 
                             rotation_range = 90,
                             preprocessing_function = add_noise,
                             validation_split = 0.1)


In [123]:
# Train

# fit the data generator to our images
datagen.fit(X_train)

# fits the model on batches with real-time data augmentation:
H = model.fit(datagen.flow(X_train, y_train, batch_size = 128), 
                            validation_data = datagen.flow(X_train, y_train, 
                                                           batch_size = 128, 
                                                           subset = "validation"),
                                                           epochs = 5)   # training for less time

Epoch 1/5


/home/ucloud/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-04-09 17:31:05.016916: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 6 of 8
2024-04-09 17:31:08.591674: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


 8/22 ━━━━━━━━━━━━━━━━━━━━ 4:14 18s/step - accuracy: 0.1111 - loss: 2.5113

KeyboardInterrupt: 

In [ ]:
# Inspect

def plot_history(H, epochs):
    plt.figure(figsize = (12,6))
    plt.subplot(1,2,1)
    plt.plot(np.arange(0, epochs), H.history["loss"], label = "train_loss")
    plt.plot(np.arange(0, epochs), H.history["val_loss"], label = "val_loss", linestyle = ":")
    plt.title("Loss curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.tight_layout()
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(np.arange(0, epochs), H.history["accuracy"], label = "train_acc")
    plt.plot(np.arange(0, epochs), H.history["val_accuracy"], label = "val_acc", linestyle = ":")
    plt.title("Accuracy curve")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.tight_layout()
    plt.legend()
    plt.show()

plot_history(H, 5)

In [ ]:
# Evaluate

predictions = model.predict(X_test, batch_size = 128)
print(classification_report(y_test.argmax(axis = 1),
                            predictions.argmax(axis = 1),
                            target_names = labelNames))

In [40]:
def main():
    folder_path = os.path.join("../../../../cds-vis-data/Tobacco3482/") # change to in for the final script
    
    list_of_images, list_of_labels = load_images(folder_path)
    
    preprocessed_images = preprocess_images(list_of_images)
    #preprocessed_images = [(preprocess_images(image), label) for image, label in preprocessed_images]
